# Test CRUD par des agents IA

## Agents

### Import des librairies

In [1]:
# Importation des modules nécessaires pour les agents, tâches, processus, Crew, et modèles LLM
from crewai import Agent, Task, Process, Crew
from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI
from pymongo import MongoClient
from bson.objectid import ObjectId
import os
import re

### Configuration d'Ollama

In [2]:
# Configuration des variables d'environnement : Pour intégrer Ollama, définissez les variables d'environnement suivantes :
os.environ["OPENAI_API_BASE"] = 'http://localhost:11434/v1' # Base URL pour l'API
os.environ["OPENAI_MODEL_NAME"] = 'gemma:2b'  # Nom du modèle utilisé
os.environ["OPENAI_API_KEY"] = 'NA' # Clé API (dans ce cas, non applicable car model en local avec Ollama)

### Chargement du model

In [3]:
# Initialisation du modèle LLM ChatOpenAI
llm = ChatOpenAI(
    model = "gemma:2b",
    base_url = "http://localhost:11434/v1")

### Définir le rôle de l'agent

In [4]:
# Création d'un agent avec un rôle, un objectif et un historique spécifique
general_agent = Agent(role = "Math Professor",
                      goal = """Provide the solution to the students that are asking mathematical questions and give them the answer.""",
                      backstory = """You are an excellent math professor that likes to solve math questions in a way that everyone can understand your solution""",
                      allow_delegation = False, # L'agent ne peut pas déléguer ses tâches
                      verbose = True, # Afficher plus de détails sur l'exécution
                      llm = llm) # Utilisation du modèle LLM configuré

### Définir la tâche de l'agent

In [5]:
# Création d'une tâche pour l'agent
task = Task(
    description="""what is pythagore theoreme ? """,
    expected_output="",  # Sortie attendue (ici, chaîne vide)
    agent=general_agent # Agent assigné à cette tâche
)

### Executer la requête

In [6]:
# Initialisation du Crew (groupe d'agents) et lancement des tâches
crew = Crew(
            agents=[general_agent], # Liste des agents
            tasks=[task], # Liste des tâches
            verbose=2 # Niveau de verbosité
        )

# Exécution des tâches par le Crew
result = crew.kickoff()

# Affichage du résultat
print(result)

 [DEBUG]: == Working Agent: Math Professor
 [INFO]: == Starting Task: what is pythagore theoreme ? 


> Entering new CrewAgentExecutor chain...
**Thought:**

The Pythagorean theorem states that in any right triangle, the square of the longest side (c) is equal to the sum of the squares of the other two sides (a and b). 

**Final Answer:**

The Pythagorean theorem states that in any right triangle, the square of the longest side (c) is equal to the sum of the squares of the other two sides (a and b).

> Finished chain.
 [DEBUG]: == [Math Professor] Task output: **

The Pythagorean theorem states that in any right triangle, the square of the longest side (c) is equal to the sum of the squares of the other two sides (a and b).


**

The Pythagorean theorem states that in any right triangle, the square of the longest side (c) is equal to the sum of the squares of the other two sides (a and b).


## CRUD

### fonctions CRUD

In [7]:
# Connexion à la base de données MongoDB
def connect_to_database():
    client = MongoClient("localhost", 27017) # Connexion au serveur MongoDB local
    db = client.test_crud_agent # Sélection de la base de données
    collection = db.crud_actions # Sélection de la collection
    return collection

In [8]:
# Création d'un nouveau cours
def create_course(course_details):
    collection = connect_to_database()
    existing_course = collection.find_one({"title": course_details["title"]})
    if existing_course:
        return f"Un cours avec le titre '{course_details['title']}' existe déjà."
    
    inserted_id = collection.insert_one(course_details).inserted_id
    return f"Le cours a été créé avec succès. ID du nouveau cours : {inserted_id}"

In [9]:
# Lecture des détails d'un cours
def read_course(course_identifier):
    collection = connect_to_database()
    
    # Rechercher par ID si l'identifiant est un ObjectId valide
    if ObjectId.is_valid(course_identifier):
        course = collection.find_one({"_id": ObjectId(course_identifier)})
        if course:
            return f"Détails du cours : {course}"
        else:
            return "Le cours n'a pas été trouvé par ID."
    
    # Rechercher par titre si l'identifiant n'est pas un ObjectId valide
    course = collection.find_one({"title": course_identifier})
    if course:
        return f"Détails du cours : {course}"
    else:
        return "Le cours n'a pas été trouvé par titre."

In [10]:
# Mise à jour des détails d'un cours
def update_course(course_identifier, new_details):
    collection = connect_to_database()
    
    # Rechercher par ID si l'identifiant est un ObjectId valide
    if ObjectId.is_valid(course_identifier):
        query = {"_id": ObjectId(course_identifier)}
    else:
        query = {"title": course_identifier}
    
    update_result = collection.update_one(query, {"$set": new_details})
    if update_result.modified_count > 0:
        return "Les détails du cours ont été mis à jour avec succès."
    else:
        return "Aucun cours n'a été mis à jour."

In [11]:
# Suppression d'un cours
def delete_course(course_identifier):
    collection = connect_to_database()
    
    # Rechercher par ID si l'identifiant est un ObjectId valide
    if ObjectId.is_valid(course_identifier):
        query = {"_id": ObjectId(course_identifier)}
    else:
        query = {"title": course_identifier}
    
    delete_result = collection.delete_one(query)
    if delete_result.deleted_count > 0:
        return "Le cours a été supprimé avec succès."
    else:
        return "Aucun cours n'a été supprimé."

### fonctions NLP

In [12]:
# Déterminer l'action à partir de l'input utilisateur
def determine_action_from_user_input(user_input):
    user_input = user_input.lower()
    if "create" in user_input:
        return "create"
    elif "read" in user_input:
        return "read"
    elif "update" in user_input:
        return "update"
    elif "delete" in user_input:
        return "delete"
    else:
        return None

In [13]:
# Extraire les détails de l'input utilisateur
def extract_details_from_user_input(user_input):
    # Utilisation des expressions régulières pour extraire les informations de l'input utilisateur
    title = re.search(r"title\s*:\s*'([^']+)'", user_input)
    instructor = re.search(r"instructor\s*:\s*'([^']+)'", user_input)
    description = re.search(r"description\s*:\s*'([^']+)'", user_input)
    course_id = re.search(r"id\s*:\s*'([^']+)'", user_input)  # Notez que j'ai corrigé l'expression régulière ici

    details = {}
    if title:
        details["title"] = title.group(1)
    if instructor:
        details["instructor"] = instructor.group(1)
    if description:
        details["description"] = description.group(1)
    if course_id:
        details["course_id"] = course_id.group(1)

    return details

In [14]:
# Gestion de la requête utilisateur
def extract_details_from_user_input(user_input):
    # Utilisation des expressions régulières pour extraire les informations de l'input utilisateur
    title = re.search(r"title\s*:\s*'([^']+)'", user_input)
    instructor = re.search(r"instructor\s*:\s*'([^']+)'", user_input)
    description = re.search(r"description\s*:\s*'([^']+)'", user_input)
    course_id = re.search(r"id\s*:\s*'([^']+)'", user_input)  # Notez que j'ai corrigé l'expression régulière ici

    details = {}
    if title:
        details["title"] = title.group(1)
    if instructor:
        details["instructor"] = instructor.group(1)
    if description:
        details["description"] = description.group(1)
    if course_id:
        details["course_id"] = course_id.group(1)

    return details

def handle_user_request(user_input):
    action = determine_action_from_user_input(user_input)
    details = extract_details_from_user_input(user_input)

    if action == "create":
        course_details = {
            "title": details.get("title", "Default Title"),
            "instructor": details.get("instructor", "Default Instructor"),
            "description": details.get("description", "Default Description")
        }
        response = create_course(course_details)
    elif action == "read":
        course_id_or_title = details.get("course_id") or details.get("title")
        if course_id_or_title:
            response = read_course(course_id_or_title)
        else:
            response = "L'ID ou le titre du cours est nécessaire pour lire les détails."
    elif action == "update":
        course_id_or_title = details.get("course_id") or details.get("title")
        if course_id_or_title:
            new_details = {
                "title": details.get("title"),
                "instructor": details.get("instructor"),
                "description": details.get("description")
            }
            new_details = {k: v for k, v in new_details.items() if v is not None}  # Remove None values
            response = update_course(course_id_or_title, new_details)
        else:
            response = "L'ID ou le titre du cours est nécessaire pour mettre à jour les détails."
    elif action == "delete":
        course_id_or_title = details.get("course_id") or details.get("title")
        if course_id_or_title:
            response = delete_course(course_id_or_title)
        else:
            response = "L'ID ou le titre du cours est nécessaire pour supprimer le cours."
    else:
        response = "Je suis désolé, je n'ai pas compris votre demande."

    return response

### Exemple de création d'un nouveau cours

In [15]:
# Creation d'un nouveau cours depuis la fonction
new_course_details = {
    "title": "Mathématiques avancées",
    "instructor": "Professeur Smith",
    "description": "Ce cours couvre des sujets avancés en mathématiques.",
    # Ajoutez d'autres détails du cours ici...
}
creation_response = create_course(new_course_details)
print(creation_response)

Le cours a été créé avec succès. ID du nouveau cours : 6669cabc32489c19304d97d5


### Exemple de lecture des détails d'un cours

In [16]:
# Lecture d'un cours depuis la fonction
course_id_to_read = "6669cabc32489c19304d97d5"  # ID du cours à lire
read_response = read_course(course_id_to_read)
print(read_response)

Détails du cours : {'_id': ObjectId('6669cabc32489c19304d97d5'), 'title': 'Mathématiques avancées', 'instructor': 'Professeur Smith', 'description': 'Ce cours couvre des sujets avancés en mathématiques.'}


### Exemple de mise à jour des détails d'un cours

In [17]:
# Mise à jour d'un cours depuis la fonction
course_id_to_update = "6669cabc32489c19304d97d5"  # ID du cours à mettre à jour
new_course_details = {
    "description": "Ce cours couvre des sujets avancés en mathématiques et en physique."
    # Autres détails mis à jour du cours...
}
update_response = update_course(course_id_to_update, new_course_details)
print(update_response)

Les détails du cours ont été mis à jour avec succès.


### Exemple de suppression d'un cours


In [18]:
# Suppression d'un cours depuis la fonction
course_id_to_delete = "6669cabc32489c19304d97d5"  # ID du cours à supprimer
delete_response = delete_course(course_id_to_delete)
print(delete_response)

Le cours a été supprimé avec succès.


### Exemple de création depuis un prompt

In [19]:
# Création d'un cours depuis un prompt
user_input = "Can you create a course with title: 'Mathématiques avancées', instructor: 'Professeur Smith', description: 'Ce cours couvre des sujets avancés en mathématiques'?"
response = handle_user_request(user_input)
print(response)

Le cours a été créé avec succès. ID du nouveau cours : 6669cac832489c19304d97da


### Exemple de lecture depuis un prompt

In [20]:
# Lecture d'un cours depuis un prompt
user_input = "Can you read the course with id: '6669cac832489c19304d97da'?"
response = handle_user_request(user_input)
print(response)

Détails du cours : {'_id': ObjectId('6669cac832489c19304d97da'), 'title': 'Mathématiques avancées', 'instructor': 'Professeur Smith', 'description': 'Ce cours couvre des sujets avancés en mathématiques'}


### Exemple de mise à jour depuis un prompt

In [21]:
# Mise à jour d'un cours depuis un prompt
user_input = "Can you update the course with title: 'Mathématiques avancées', description: 'Ce cours couvre des sujets avancés en mathématiques et en physique'?"
response = handle_user_request(user_input)
print(response)

Les détails du cours ont été mis à jour avec succès.


### Exemple de suppresion depuis un prompt

In [22]:
# Suppression d'un cours depuis un prompt
user_input = "Can you delete the course with title: 'Mathématiques avancées'?"
response = handle_user_request(user_input)
print(response)

Le cours a été supprimé avec succès.
